In [ ]:
# Import required libraries
import numpy as np
import plotly.graph_objects as go
import seaborn as sns

# Import helper functions
import layout_helpers as helpers

## Configuration Parameters

Define all input parameters for the experimental setup.

In [ ]:
# ==================== INPUT PARAMETERS ====================

# Wing geometry file
wing_shape_dir = "./Airfoil_scan.dat"

# Wing dimensions
chord = 2  # meters

# Sensor patch positions (one entry per patch)
target_real_positions = [0.05, 0.1, 0.55]  # Chordwise positions for each patch
target_sides = ['pressure', 'suction', 'suction']  # Surface side: 'pressure' or 'suction'
isens0_baros = [12, 12, 12]  # Index of the reference sensor for each patch (0-indexed)

# ==========================================================

In [ ]:
# Fixed Aerosense sensor specifications (do not modify)
nbaros = 24  # Number of sensors per patch
sensor_spacing = 14.0  # mm - distance between each sensor
sensor_thickness = 4.0  # mm - thickness of the sensors
limit_before = 36.0  # mm - distance before first sensor for physical limit marker
limit_after = 42.0  # mm - distance after last sensor for physical limit marker

## Wing Geometry Creation

Create the wing geometry from the airfoil coordinates file.

In [ ]:
# Create wing geometry using configuration parameters
zwing, lwing, twing, nwing, blade_properties = helpers.make_labbook(
    chord=chord,
    span=1,
    wing_shape_dir=wing_shape_dir,
)

print(f"Wing created with {len(zwing)} points")

# Plot the wing to visualize
wing_trace = helpers.make_go_wing(zwing)
layout = go.Layout(
    title="Wing Geometry",
    showlegend=False,
    xaxis=dict(scaleanchor="y", scaleratio=1, title="Chordwise position"),
    yaxis=dict(title="Thickness"),
    width=800,
    height=400
)
fig = go.Figure(data=[wing_trace], layout=layout)
fig.show()

## Aerosense Sensor Positioning

In [ ]:
# Position all sensor patches using the helper function
zBaros_list, limit_points, zsens0_baros, zBaros_all, db = helpers.position_aerosense_sensors(
    zwing=zwing,
    nwing=nwing,
    chord=chord,
    target_real_positions=target_real_positions,
    target_sides=target_sides,
    isens0_baros=isens0_baros,
    nbaros=nbaros,
    sensor_spacing=sensor_spacing,
    sensor_thickness=sensor_thickness,
    limit_before=limit_before,
    limit_after=limit_after,
)

print(f"Number of sensor patches: {len(zsens0_baros)}")
print(f"Sensors per patch: {nbaros}")
print(f"Total sensors: {len(zBaros_all)}")

## Visualization of Complete Sensor Layout

Plot the wing with all sensor patches and their physical limits.

In [ ]:
# Create color palette for different sensor patches
colors = sns.color_palette("Set2", len(zBaros_list))

# Create traces for both sensors and their limit markers
aerosense_traces = []
limit_traces = []

for i, zBaros in enumerate(zBaros_list):
    # Convert color to RGB string
    color_rgb = f"rgb({colors[i][0]*255:.0f},{colors[i][1]*255:.0f},{colors[i][2]*255:.0f})"
    
    # Sensor trace
    aerosense_trace = go.Scatter(
        x=zBaros.real,
        y=zBaros.imag,
        mode="markers",
        marker=dict(size=8, symbol="diamond", color=color_rgb),
        name=f"Patch {i+1} sensors"
    )
    aerosense_traces.append(aerosense_trace)

    # Limit markers for this patch (2 points: start and end)
    patch_limits = [limit_points[i*2], limit_points[i*2+1]]
    limit_trace = go.Scatter(
        x=[p.real for p in patch_limits],
        y=[p.imag for p in patch_limits],
        mode="markers",
        marker=dict(size=6, symbol="cross", color=color_rgb),
        name=f"Patch {i+1} limits"
    )
    limit_traces.append(limit_trace)

# Create wing trace for plotting
wing_trace = helpers.make_go_wing(zwing)
layout = go.Layout(
    title="Aerosense Sensor Layout on wing",
    showlegend=True,
    xaxis=dict(scaleanchor="y", scaleratio=1, title="Chordwise position"),
    yaxis=dict(title="Thickness"),
    width=1200,
    height=800
)
fig = go.Figure(data=[wing_trace] + aerosense_traces + limit_traces, layout=layout)
fig.show()

# Save the plot as PNG
fig.write_image("Aerosense_layout.png", width=1200, height=800, scale=2)
print("Plot saved as 'Aerosense_layout.png'")